In [1]:
import openai
import langchain
import pinecone
import os
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import openai

/Users/abhinaymannepalli/Desktop/llmproject/env/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [73]:
###Reading the Documents from the directory
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents


In [74]:
docs=read_doc('documents/')
docs

[Document(metadata={'source': 'documents/preowned_cars_dashboard.pdf', 'page': 0}, page_content='Pre-owned Cars Dashboard\nToshitt Ahuja\nRutgers University\nPiscataway, NJ, USA\ntoshitt.ahuja@rutgers.eduAbhinay Mannepalli\nRutgers University\nPiscataway, NJ, USA\nam2977@scarletmail.rutgers.eduPavankumar Prakash Savanur\nRutgers University\nPiscataway, NJ, USA\nps1149@scarletmail.rutgers.edu\nAbstract —We have worked on building a multiple featured, mul-\ntilevel, highly interactive dashboard that allows users to demonstrate\nand get an idea of car prices in different states of the US. The data[1]\nwas scraped from Craigslist and was open-sourced at Kaggle. We\nhave visualized the features of data through various plots and maps\nprovided by Plotly[2] and Dash[3], python libraries based on concept\nof visualization.\nKeywords— Plotly, Dash, Choropleth Maps\nI. P ROJECT DESCRIPTION\nSearching for the perfect car, is quite a task. To strike a cordial\nbalance between the price and require

In [76]:
###Divide the documents or pages in to chunks
def chunk_data(docs,chunk_size,chunk_overlap):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

In [77]:
documents=chunk_data(docs=docs,chunk_size=500,chunk_overlap=50)
len(documents)
chunk_data=[documents[i].page_content for i in range(len(documents))]
chunk_data

22

In [108]:
###OpenAI Embeddings
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x12c842b30>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x12efa6770>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='sk-Wc9EYSUZyrHywmkkughKtzvojskIHjzu0RrJv0ncwQT3BlbkFJulFFiCc0XHzKjJ0afgMJZYGTkZF2BT4h6H-pElRLAA', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

In [95]:
import tqdm
from tqdm import tqdm
import ast

In [109]:
####Upload the embeddings to Pincone Vector Database
from pinecone import Pinecone

pc = Pinecone(os.environ['PINECONE_KEY'])
index_name="langchainvector"

chunk_embeddings=embeddings.embed_query

vectors=[]
for index, content in tqdm(enumerate(chunk_data),total=len(chunk_data)):
    text=str(content)
    # # print(str(text))
    # print(embeddings.embed_query(str(text)))
    # text_embeddings=embeddings.embed_query(text)

    vectors.append({
        "id": str(index),
        "values": embeddings.embed_query(str(text)),
        "metadata": {'text': text}
    })

###Upload the vectors in PineCone
index=pc.Index(host=os.environ['PINECONE_INDEX'])
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

  0%|          | 0/22 [00:00<?, ?it/s]

[-0.010489045095423878, -0.0024172636816942213, 0.012737185030168168, -0.023848055549789707, -0.010618877043491364, 0.015716484859701618, -0.015866815216567617, -0.012962682428112217, -0.026417360194999663, -0.03599758258527366, 0.010140548862786022, -0.009457223555580194, 0.009546055892083077, 0.015648151863319772, 0.011350035075635473, -0.006447175181526449, 0.010550544047109518, -0.014855493948167497, -0.005347021110962182, -0.02875433339756936, -0.020103433052566275, 0.013946671243017619, -0.0066145897188104345, 0.01944744113017769, -0.02175708001660762, -0.011999193884650379, 0.01299001581292946, -0.011657530765386202, 0.007994907491291975, -0.01959777148704369, 0.014705162659978973, -0.007461913472274671, -0.0124433555671648, -0.03364694315595661, -0.03326428135645035, 0.00034892304028925084, 0.004486031130750586, 0.0001647454833396225, 0.0246680459184367, 0.006266093951833841, 0.021388084443848727, 0.002065350892369525, -0.00359429167469617, -0.02439471579555437, -0.003645540956

  9%|▉         | 2/22 [00:00<00:09,  2.21it/s]

[0.005289591458538811, 0.0010531352750101114, 0.0177571533914677, -0.02351086992733885, -0.0027262829632493848, 0.015418822165426482, -0.0015500309166575814, -0.009204526562976354, -0.03120610907423229, -0.037753437252205764, -0.01410793905685303, 0.03616620651742041, -0.022958172812269865, 0.006781163722016327, -0.0067634490350722215, 0.0044959756254848385, 0.011677489782321807, -0.02841428148989294, -0.013498555129827427, -0.023765959184159757, -0.03883048947334931, 0.004846725216258762, 0.00761729489686841, -0.0004791809838071956, -0.0028396565871626244, 0.009856425366138778, 0.027294716255257753, -0.03199972257897979, -0.004035394696260685, -0.0006682847838119288, 0.011649146842004791, -0.019769536612911587, -0.012541964828813496, -0.016184093661179556, -0.0250697567904846, -0.010359521171499748, 0.013200949134224526, 0.0008454311293840277, 0.016425013776148426, 0.011330283408333482, 0.0033268093139722896, -0.004807753091246248, -0.013278893384249554, -0.030185744596367955, -0.0012

 14%|█▎        | 3/22 [00:01<00:08,  2.34it/s]

...

[7.526885854087573e-05, 0.010126163395482462, 0.004694269571053455, -0.030695539403703034, -0.015050132041294207, -0.00821092736459053, -0.018000630917441588, 0.0030006443110750748, -0.028003856937361456, -0.02488512767423708, -0.017081834618408422, 0.03745063074956339, -0.002623744113373206, -0.014377211424708812, -0.013367830034169428, 0.019967630789088232, 0.020058214949271864, 0.0017745050387967232, 0.0039922320850818436, 0.006975082779624301, -0.00821739819393084, 0.01433838924263472, -0.01656420366144199, 0.01988998456229487, -0.001311872056686603, 0.019268827786464188, 0.0016135539855816057, -0.010993195853760913, -0.006664503926047667, 0.0012673880153550616, 0.006421864123931345, 0.00045494945434512, -0.03408602673531383, 0.005991583309462273, -0.02127464890886234, -0.0016337740467015148, -0.008456802115715712, -0.006192165049173053, 0.0312390536780307, -0.015813637749385823, 0.015050132041294207, -0.0022581669356942914, 0.007919759602050942, -0.019165301967599937, -0.007686826

 95%|█████████▌| 21/22 [00:08<00:00,  2.55it/s]

[-0.011180699431038554, -0.014470697173320743, 0.008963094541373725, -0.030503648928178185, 0.007076475657691376, -0.0182174565072417, -0.0012949817530554105, -0.009505911274535305, -0.04165124729061354, -0.02022984911393509, -0.00566316657164284, 0.024413510515920042, 0.00857253136165086, -0.0038559841524449896, 0.010214220312196159, 0.016469851960276196, 0.024784214172068136, 0.004865490609218136, 0.007692109246976518, -0.007241968427851962, -0.004885349667131596, 0.015529852206200161, 0.005927955097032039, -0.0126966141929115, -0.005659856651047045, 0.013159993763096615, 0.00883070051150978, -0.028941394346641485, 0.017025907444498334, 0.014285345345246695, 0.007215489528746911, -0.014761965529137614, -0.026094918513614885, -0.0018270414771113111, -0.030027027812964645, -0.009929572728893499, 0.0026942241422331275, 0.001128661502745876, 0.016536048509546857, -0.017171542088068082, 0.02132872226264143, 0.001460474639631324, 0.0016797528045662882, -0.007930418873260667, -0.01009506596

100%|██████████| 22/22 [00:08<00:00,  2.62it/s]

[-0.012440228519141535, -0.014841465082699862, 0.027708734825760934, -0.01342732525763831, -0.01400838302863519, -0.010080995537099863, 0.019209896233757105, -0.012349218741323104, -0.006167609084182983, -0.035115467259815035, -0.00145439356778922, 0.03900785192197359, 0.00963295112543807, 0.014421423679597587, -0.01209719445825531, 0.0018569333233724528, 0.002479994792301315, -0.030074968559942485, -0.0009022140739923341, -0.027008667062353966, 0.006048597229127655, -0.00495298883246275, -0.009506938518242863, -0.012356219493462984, 0.0042319178813135265, 0.0073507254856124485, 0.020147989569919967, -0.02497846570559614, 0.016521631626431586, -0.019433918439588003, 0.011222107891351363, -0.012818265409404535, -0.00515250887146538, -0.011733158140949449, -0.01916789172091783, -0.013861368165020345, -0.005320525060177242, -0.004025397555832328, 0.013581340873393036, 0.010998085685520466, -0.004711465211943466, 0.016003579693371, -0.018915866506527414, -0.01325230831678657, 0.00767275729